In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [9]:
# 데이터 준비
(X_train, y_train), (X_test, y_test) = tf_keras.datasets.imdb.load_data(num_words=1000) # 1000 개의 단어 집합 사용

In [11]:
print(type(X_train))
print(X_train.shape) # (25000, ) : 25000개의 문장 의미
print(type(X_train[0])) # 각 문장은 숫자(단어 번호)의 리스트
print(X_train[0])
print(len(X_train[0]), len(X_train[1])) # 각 문장은 서로 다른 크기의 단어 집합 (one-hot-encoding 이 아님)
print(np.unique(y_train, return_counts=True)) # 긍정 vs 부정

<class 'numpy.ndarray'>
(25000,)
<class 'list'>
[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
218 189
(array([0, 1]), array([12500, 12500]))


In [12]:
# 단어 사전 확인

word_to_index = tf_keras.datasets.imdb.get_word_index()

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [17]:
print( type( word_to_index ) )
print( list( word_to_index.keys() )[:10] )
print( list( word_to_index.values() )[:10] )

<class 'dict'>
['fawn', 'tsukino', 'nunnery', 'sonja', 'vani', 'woods', 'spiders', 'hanging', 'woody', 'trawling']
[34701, 52006, 52007, 16816, 63951, 1408, 16115, 2345, 2289, 52008]


In [19]:
# 단어 번호로 단어를 찾기 위한 맵 만들기
index_to_word = { value: key for key, value in word_to_index.items() }

print( type( index_to_word ) )
print( list( index_to_word.keys() )[:10] )
print( list( index_to_word.values() )[:10] )

<class 'dict'>
[34701, 52006, 52007, 16816, 63951, 1408, 16115, 2345, 2289, 52008]
['fawn', 'tsukino', 'nunnery', 'sonja', 'vani', 'woods', 'spiders', 'hanging', 'woody', 'trawling']


In [26]:
print( len(index_to_word.keys()) )
print( index_to_word[88583] )
# print( index_to_word[88585] ) # 없는 키의 값을 요청하면 오류
print( index_to_word.get(88585) ) # get을 사용하면 없는 키의 값을 요청했을 때 None 반환
print( index_to_word.get(88585, '?') ) # get을 사용하면 없는 키의 값을 요청했을 때 '?' 반환

88584
voorhees'
None
?


In [28]:
print( word_to_index.get('?') )

None
